In [1]:
import tensorflow as tf
from keras.models import load_model

q_eval = load_model('humvee_largestt_7_checkpoint_7.0.h5')

In [2]:
def filter_image(image):
    # If the image has 4 channels (RGBA), ignore the alpha channel
    if image.shape[2] == 4:
        image = image[:, :, :3]
    
    # Define the color masks for green and blue
    green_mask = (image[:, :, 0] == 0) & (image[:, :, 1] == 255) & (image[:, :, 2] == 0)
    blue_mask = (image[:, :, 0] == 255) & (image[:, :, 1] == 0) & (image[:, :, 2] == 0)
    
    # Create a black image (same shape as the input image)
    black_image = np.zeros_like(image)
    
    # Apply the masks to retain only green and blue pixels
    black_image[green_mask] = [0, 255, 0]
    black_image[blue_mask] = [255, 0, 0]
    
    return black_image

In [3]:
import numpy as np

def get_humvee_position(image):
    """
    Get the position of the blue dot (humvee) in the image.
    
    Parameters:
    - image (np.ndarray): The input image in which to find the blue dot.

    Returns:
    - tuple: A tuple containing the (x, y) coordinates of the blue dot,
      or None if the blue dot is not found.
    """
    # Define the blue color mask
    blue_mask = (image[:, :, 0] == 255) & (image[:, :, 1] == 0) & (image[:, :, 2] == 0)
    blue_coords = np.column_stack(np.where(blue_mask))
    
    if blue_coords.size == 0:
        # If no blue dot is found, return None
        return None
    
    # Use the average position if there are multiple blue pixels
    humvee_pos = np.mean(blue_coords, axis=0)
    
    return (humvee_pos[1], humvee_pos[0])  # Return as (x, y)

In [4]:
import cv2
import numpy as np

def get_state(image, prev_input):
    # Convert the image to grayscale for easier processing
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Find the blue dot position (humvee)
    blue_mask = (image[:, :, 0] == 255) & (image[:, :, 1] == 0) & (image[:, :, 2] == 0)
    blue_coords = np.column_stack(np.where(blue_mask))
    
    if blue_coords.size == 0:
        # If no blue dot is found, return a default state
        return np.zeros(9+6, dtype=np.float32)
    
    humvee_pos = np.mean(blue_coords, axis=0)  # Use the average position if there are multiple pixels

    # Find the green dot positions
    green_mask = (image[:, :, 0] == 0) & (image[:, :, 1] == 255) & (image[:, :, 2] == 0)
    green_coords = np.column_stack(np.where(green_mask))

    if green_coords.size == 0:
        diff_x = 0
        diff_y = 0
    else:
        # Calculate distances to all green dots and find the closest one
        distances = np.linalg.norm(green_coords - humvee_pos, axis=1)
        closest_green_pos = green_coords[np.argmin(distances)]
        
        # Calculate the differences in x and y coordinates
        diff_x = (closest_green_pos[1] - humvee_pos[1]) / image.shape[1]
        diff_y = (closest_green_pos[0] - humvee_pos[0]) / image.shape[0]
    
    # Squish the humvee position coordinates between 0 and 1
    humvee_x = humvee_pos[1] / image.shape[1]
    humvee_y = humvee_pos[0] / image.shape[0]
    
    last_input = np.zeros(9, dtype=np.float32)
    last_input[prev_input] = 1.0

    return [
        *last_input,
        humvee_x,
        humvee_y,
        1.0-humvee_x,
        1.0-humvee_y,
        diff_x / 1.5,
        diff_y / 1.5
        ]

In [5]:
import pyautogui
import keyboard

region = {'top': 776, 'left': 57, 'width': 321 - 57, 'height': 1039 - 776}
def move_humvee(direction_index, humvee_pos):
    direction_vectors = {
        0: (0, 0),      # No movement
        1: (0, 1),      # North
        2: (1, 1),      # Northeast
        3: (1, 0),      # East
        4: (1, -1),     # Southeast
        5: (0, -1),     # South
        6: (-1, -1),    # Southwest
        7: (-1, 0),     # West
        8: (-1, 1)      # Northwest
    }

    # Get the direction vector
    direction_vector = direction_vectors.get(direction_index, (0, 0))

    # Calculate the target position
    offset = 20
    target_x = humvee_pos[0] + direction_vector[0] * offset
    target_y = humvee_pos[1] + direction_vector[1] * offset
    
    x_min = region['left']
    x_max = region['left'] + region['width']
    y_min = region['top']
    y_max = region['top'] + region['height']

    # Constrain target_x and target_y to be within the region
    target_x = max(x_min, min(target_x, x_max - 1))
    target_y = max(y_min, min(target_y, y_max - 1))

    # Perform the right-click action
    if direction_index != 0:
        pyautogui.rightClick(x=target_x, y=target_y)
    else:
        keyboard.press_and_release('s')
    pyautogui.leftClick(x=humvee_pos[0], y=humvee_pos[1])
    pyautogui.moveTo(20, 20)
    print(f"Right-clicked at position: ({target_x}, {target_y})")

# Example usage (you should call this function with the actual index):
# move_humvee(max_index)


In [6]:
import time
import keyboard
import mss

# Initialize a variable to control the while loop
running = True
paused = False

def toggle_pause():
    global paused
    paused = not paused
    if paused:
        print("Loop paused.")
    else:
        print("Loop resumed.")

def exit_program():
    global running
    running = False
    print("Exiting program.")

keyboard.add_hotkey('F6', toggle_pause)
keyboard.add_hotkey('F7', exit_program)

print("Program started. Press F6 to pause/resume and F7 to exit.")

prev_action = 0

with mss.mss() as sct:
    while running:
        if not paused:
            screenshot = sct.grab(region)
            screenshot_np = np.array(screenshot)

            # Apply the filter to the captured image
            filtered_image = filter_image(screenshot_np)

            # Get the state
            state = get_state(filtered_image, prev_action)
            formatted_state = tf.expand_dims(tf.constant(state, dtype=tf.float32), axis=0)
            action = q_eval.predict(formatted_state, verbose=False)
            action_values = action.flatten()  # Flatten in case it's a 2D tensor (batch size, num_actions)

            # Find the index of the maximum value
            max_index = tf.argmax(action_values).numpy()  # or use action_values.argmax() for NumPy array
            prev_action = max_index
            humvee_pos = get_humvee_position(filtered_image)
            if (humvee_pos):
                move_humvee(max_index, [humvee_pos[0]+region['left'], humvee_pos[1]+region['top'] ])

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        

cv2.destroyAllWindows()

print("Program exited.")


Program started. Press F6 to pause/resume and F7 to exit.
Right-clicked at position: (126.5, 1027.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at position: (126.5, 907.5)
Right-clicked at 

KeyboardInterrupt: 